# Get BBC Dataset

In [0]:
from google.colab import drive
drive.mount("./gdrive")

Drive already mounted at ./gdrive; to attempt to forcibly remount, call drive.mount("./gdrive", force_remount=True).


In [0]:
cd gdrive/My\ Drive/Colab\ Notebooks/Datasets/Text\ Summarization

/content/gdrive/My Drive/Colab Notebooks/Datasets/Text Summarization


In [0]:
import pickle 
with open('myfile.pkl','rb') as fp:
  heading, article = pickle.load(fp)

**Data from BBC News Dataset Loaded. The variables are *heading* and *article***

# Preprocess the Data , remove the \n\n and \xa0 , lower case everything  (stemming/lemmatization advantageous? Try it out later)

## I think I should keep the comma's in for now , maybe the Attention mechanism later would find words near a comma a good place to give attention to

In [0]:
heading = [h.lower() for h in heading]
article = [a.lower() for a in article]

In [0]:
article = [a.replace('\n','').replace('\xa0',' ') for a in article]

# Toeknize the data , Make Word Embeddings from GloVe

## Mainly what I have to do is make a matrix of size (vocab_size, 100) which I can initialize as my weight matrix. Be careful, for now I am assuming no new words can appear in a text other than the ones I have seen before , so later if this works improve it to take in other words too

In [0]:
word_dimensions = 100
import pandas as pd
df = pd.read_csv('glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)

In [0]:
glove = {key: val.values for key, val in df.T.items()}

In [0]:
import nltk
from collections import Counter
import numpy as np

In [0]:
words = []
tokenizer = nltk.tokenize.WordPunctTokenizer()
for a in article:
  tokenized_text = tokenizer.tokenize(a)
  for tokens in tokenized_text:
    words.append(tokens)
    
for h in heading:
  tokenized_text = tokenizer.tokenize(h)
  for tokens in tokenized_text:
    words.append(tokens)

In [0]:
wordcounts = Counter(words).most_common(170000)
vocab_list = np.array([word for word,_ in wordcounts])
dictionary_word_code = {word:code for code,word in enumerate(vocab_list)}

In [1]:
embedding_list = []
i = 0
while i != vocab_list.shape[0] :
  try:
    embedding_list.append(glove[vocab_list[i]])
    i = i+1
  except KeyError as e:
    embedding_list.append(np.random.normal(0,0.1,[word_dimensions]))
    i = i+1
    
  if i%1000 == 0:
    print(i)

In [0]:
embedding_matrix = np.vstack(embedding_list)

**Embedding Matrix is created and in the variable *embedding_matrix***

# Making a Bi-Directional RNN with Attention , no dropout layer

## For now making a 3 Layer LSTM deep network (could cause vanishing gradient problem) and not using the EOS thing, will make the summary a fixed number of words , mostly wont be coherent but lets see if any output comes at all first.

In [0]:
import tensorflow as tf

#### 'The' also has a code of 0 and you are gonna zero pad things so could lead to problems

#### Also you are using too much RAM afterwards change variable names so you overwriting them and not keep making new ones
Ok so for saving RAM I am thinking I will make a pickle file for all the important stuff and load it in so all the useless variables dont take up space

In [0]:
tokenized_articles = []
tokenized_headings = []
for a in article:
  tokens = tokenizer.tokenize(a)
  tokenized_articles.append(tokens)
for h in heading:
  tokens = tokenizer.tokenize(h)
  tokenized_headings.append(tokens)
  
max_length_article = max([len(a) for a in tokenized_articles])
max_length_heading = max([len(h) for h in tokenized_headings])

numerical_articles = []
numerical_headings = []
for a in tokenized_articles:
    list1 = [dictionary_word_code[b] for b in a]
    numerical_articles.append(list1)
for h in tokenized_headings:
  list1 = [dictionary_word_code[b] for b in h]
  numerical_headings.append(list1)
  
padded_numerical_articles = []
padded_numerical_headings = []
for a in numerical_articles:
  zeros = [0]*(max_length_article - len(a))
  list1 = a + zeros
  padded_numerical_articles.append(list1)
for h in numerical_headings:
  zeros = [0]*(max_length_heading - len(h))
  list1 = h + zeros
  padded_numerical_headings.append(list1)
  
with open("input_data.pickle", "wb") as f:
    pickle.dump((embedding_matrix,padded_numerical_articles,padded_numerical_headings,vocab_list), f)

# Too much RAM was used so start from here

In [6]:
import pickle 
with open("input_data.pickle","rb") as f:
  _ , articles , headings, vocab_list = pickle.load(f)

In [7]:
import numpy as np
def next_batch(articles,headings,batch_size,num_time_steps = 850,num_output_steps = 84):
  
  snipped_articles = []
  snipped_headings = []
  
  for a in articles:
    part_article = a[0:num_time_steps]
    snipped_articles.append(part_article)
    
  random = np.random.randint(0,len(articles) - batch_size -1)
  batch_x = np.array(snipped_articles[random:random+batch_size]).reshape([batch_size,num_time_steps,1])
  onehotlistarticles = []
  for article in batch_x:
    for num in article:
      zeros = np.array([0]*len(vocab_list))
      zeros[num] = 1
      onehotlistarticles.append(zeros)
  onehotlistarticles = np.array(onehotlistarticles).reshape([batch_size,num_time_steps,-1])
  
  for h in headings:
    part_heading = h[0:num_output_steps]
    snipped_headings.append(part_heading)
  
  batch_y = np.array(snipped_headings[random:random+batch_size]).reshape([batch_size,num_output_steps,1])
  onehotlistheadings = []
  for heading in batch_y:
    for num in heading:
      zeros = np.array([0]*len(vocab_list))
      zeros[num] = 1
      onehotlistheadings.append(zeros)
  onehotlistheadings = np.array(onehotlistheadings).reshape([batch_size,num_output_steps,-1])
  
  return onehotlistarticles , onehotlistheadings

# TensorFlow Graph 

## Encoder 

In [8]:
def length(sequence):
  used = tf.reduce_max(sequence, 2)
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

**See Imext Notebook for explaination of tf.reduce_max and how axis collapses and stuff**

In [9]:
import tensorflow as tf
import numpy as np

/home/administrator/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
num_time_steps = 850
num_output_steps = 84
dimensions_rnn_cell = 100
batch_size = 2

In [11]:
tf.reset_default_graph()

In [12]:
X = tf.placeholder(tf.float32,shape = [None,num_time_steps,len(vocab_list)])

** Right now too tired, not initializing with Embedding Matrix as its a pain , have no clue what the fuck the function wants from me, come back later here if the performance is not so good , see the documentation for initializer in LSTMCell**

In [13]:
lstm_fw_cell = [tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)]
lstm_bw_cell = [tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell),tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell)]

outputs, output_state_fw, output_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, X, dtype='float32', sequence_length = length(X))

** Cool so session crashes after using all the RAM**

** For testing if this is working or not **

In [10]:
with tf.Session() as sess:
  sess.run(init)
  batch_x, v = next_batch(articles,headings,batch_size)
  a1 = sess.run(outputs,feed_dict = {X:batch_x})

In [17]:
a1.shape

(3, 850, 200)

## Attention Mechanism : For now, wont be taking input from the final output of the network and lets see how it performs, afterwards take as input from the output of the final stage too 

## I think thats possible because the final outputs are also placeholders right? So that can be input to your 'neural_network' function

### Define a Neural Network basically 200 : 100 : 84

In [14]:
combined_BiRNN_outputs = tf.placeholder(tf.float32,shape = [None,num_time_steps,2*dimensions_rnn_cell])

In [18]:
num_hidden_neurons = 100
def neural_network(combined_BiRnn_output):
    
    Wx1 = tf.Variable(tf.random_normal([batch_size,2*dimensions_rnn_cell,num_hidden_neurons],0.3,1/200))
    Wx2 = tf.Variable(tf.random_normal([batch_size,num_hidden_neurons,num_output_steps],0.5,1/50))
    b1 = tf.Variable(tf.constant(0.5,shape = [num_hidden_neurons]))
    b2 = tf.Variable(tf.constant(0.3,shape = [num_output_steps]))
    first_out = tf.nn.tanh(tf.add(tf.matmul(combined_BiRnn_output,Wx1),b1))
    e = tf.nn.relu(tf.add(tf.matmul(first_out,Wx2),b2))
        
    return e

In [19]:
def softmax(e_values):
    j = tf.exp(e_values)
    totals = tf.reduce_sum(j,1)
    j = tf.unstack(j,axis = 0)
    totals = tf.unstack(totals, axis = 0)
    for i in range(batch_size):
        j[i] = j[i]/totals[i]
    j = tf.stack(j,axis = 0)
    return j

In [20]:
e_vals = neural_network(combined_BiRNN_outputs)
alpha_values = softmax(e_vals)

In [21]:
init = tf.global_variables_initializer()

** To test whether alpha values are coming or not **

In [22]:
with tf.Session() as sess:
    sess.run(init)
    batch_x, _ = next_batch(articles,headings,batch_size)
    outs = sess.run(outputs,feed_dict = {X: batch_x})
    
    alpha = sess.run(alpha_values,feed_dict = {combined_BiRNN_outputs:outs})

** Weighted Average **

In [1]:
def weighted_average(outputs,alphas):
    
    alphas = tf.transpose(alphas,perm = [0,2,1])
    context_vectors = tf.matmul(alphas,outputs)
    
    return context_vectors

In [2]:
context_vectors = weighted_average(outputs,alpha_values)

NameError: name 'outputs' is not defined

### Another idea you could do is instead of one hot encoding the input words use the embedding matrix you created, though wont be as useful if you could load up the embedding matrix as the weight vector it may be better than normal one hot encoding

### Or use a LSTM cell as the first layer and top it off with 2 BiRnn layers or something if LSTM cell supports initialzing 


In [3]:
context = tf.placeholder(tf.float32,shape = [batch_size,num_output_steps,dimensions_rnn_cell])

NameError: name 'tf' is not defined

In [4]:
cell_factory = tf.nn.rnn_cell.LSTMCell(dimensions_rnn_cell,activation=tf.nn.relu)
initial_state = cell_factory.zero_state(batch_size, dtype=tf.float32)
decoder, _ = tf.nn.dynamic_rnn(cell_factory,context,initial_state=initial_state)

NameError: name 'tf' is not defined

In [5]:
def output_neural_network(decoder_rnn):
    
    Wx1 = tf.Variable(tf.random_normal([batch_size,100,200000],0.5,1/num_output_steps))
    b1 = tf.Variable(tf.constant(0.5,shape = [200000]))
    first_out = tf.nn.softmax(tf.nn.relu((tf.add(tf.matmul(decoder_rnn,Wx1),b1))))
        
    return first_out

In [6]:
probability_distribution = output_neural_network(decoder)

NameError: name 'decoder' is not defined

In [7]:
c_e = tf.nn.softmax_cross_entropy_with_logits_v2(logits = probability_distribution,labels = batch_y)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.3)
train = optimizer.minimize(c_e)

NameError: name 'tf' is not defined